https://twitter.com/taketo1024_2/status/1585942690495336449

In [1]:
using LinearAlgebra: LinearAlgebra
using SparseArrays
using Polynomials

In [2]:
Base.promote_op(LinearAlgebra.matprod, Polynomial{Rational{Int}, :h}, Polynomial{Rational, :h})

Any

これ↑が以下のようになる理由.

In [3]:
p = Polynomial(Rational[0, 1], :h)

Polynomial(h)

In [4]:
eltype(p)

Rational

In [5]:
typeof(p)

Polynomial{Rational, :h}

`Rational` は非具象型なので `Polynomial{Rational, :h}` 型のオブジェクトを使用すると型不安定になって計算速度が劣化する危険性があるので要注意.

Juliaでのプログラミングでは具象型の情報が正しく伝搬するように書くことが最も重要である.

非具象型と具象型の区別は非常に基本的. 

非具象型を避けるべき場合があることについては公式ドキュメントの

* https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-abstract-container

を読めばわかる.

Juliaの大きな特徴の1つは, 本質的に多重ディスパッチを使ったコードによって, 数値の型が混合している場合の演算を定義していることである.  この [mixed type arithmetic 実装](https://www.google.com/search?q=mixed-type+site%3Adocs.julialang.org)は多重ディスパッチを採用していない場合にはかなりやっかいな問題になる.

数値の型が混合している場合の演算時の型のプロモーションのルールはJuliaのコードできちんと記述されているので, Juliaの型の取り扱いは実際には全然「ゆるふわ」ではない.

むしろ「ゆるふわ」でないせいで面倒なことになる場合が多いように思われる.

In [6]:
a = sparse([p;;])

1×1 SparseMatrixCSC{Polynomial{Rational, :h}, Int64} with 1 stored entry:
 Polynomial(h)

In [7]:
-a

1×1 SparseMatrixCSC{Polynomial{Rational{Int64}, :h}, Int64} with 1 stored entry:
 Polynomial(-h)

`-a` の `eltype` が `Polynomial{Rational, :h}` ではなく具象型の `Polynomial{Rational{Int64}, :h}` になっている.

In [8]:
(-a)*a

1×1 SparseMatrixCSC{Any, Int64} with 1 stored entry:
 Polynomial(-h^2)

`(-a)*a` の `eltype` が `Polynomial{Any, :h}` になってしまった!

こうなってしまった理由はすでに説明したように次のように型がプロモートするようになっているからである.

In [9]:
Base.promote_op(LinearAlgebra.matprod, Polynomial{Rational{Int}, :h}, Polynomial{Rational, :h})

Any

ただし, `Any` が出て来るより前に非具象型の `Rational` を `eltype` とする多項式を使ってしまっている時点で計算速度的に損失が生じていたと考えられる.

こうならないようにするためには, `p = Polynomial(Rational[0, 1], :h)` を次に置き換えればよい(`Rational` → `Rational{Int}`).

In [10]:
q = Polynomial(Rational{Int}[0, 1], :h)

Polynomial(h)

In [11]:
typeof(q)

Polynomial{Rational{Int64}, :h}

In [12]:
eltype(q)

Rational{Int64}

In [13]:
isconcretetype(Rational) # Rationalは具象型ではない

false

In [14]:
isconcretetype(Rational{Int}) # Rational{Int}は具象型である

true

In [15]:
b = sparse([q;;])

1×1 SparseMatrixCSC{Polynomial{Rational{Int64}, :h}, Int64} with 1 stored entry:
 Polynomial(h)

In [16]:
-b

1×1 SparseMatrixCSC{Polynomial{Rational{Int64}, :h}, Int64} with 1 stored entry:
 Polynomial(-h)

In [17]:
(-b)*b

1×1 SparseMatrixCSC{Polynomial{Rational{Int64}, :h}, Int64} with 1 stored entry:
 Polynomial(-h^2)

In [18]:
Base.promote_op(LinearAlgebra.matprod, Polynomial{Rational{Int}, :h}, Polynomial{Rational{Int}, :h})

Polynomial{Rational{Int64}, :h}

In [19]:
qq = Polynomial([0, 1//1], :h)

Polynomial(h)

In [20]:
typeof(qq)

Polynomial{Rational{Int64}, :h}

In [21]:
qq == q

true

他の例も示しておこう.

In [22]:
r = Polynomial(Rational{BigInt}[0, 1], :h)

Polynomial(h)

In [23]:
typeof(r)

Polynomial{Rational{BigInt}, :h}

In [24]:
c = sparse([r;;])

1×1 SparseMatrixCSC{Polynomial{Rational{BigInt}, :h}, Int64} with 1 stored entry:
 Polynomial(h)

In [25]:
(-b)*c

1×1 SparseMatrixCSC{Polynomial{Rational{BigInt}, :h}, Int64} with 1 stored entry:
 Polynomial(-h^2)

In [26]:
Base.promote_op(LinearAlgebra.matprod, Polynomial{Rational{Int}, :h}, Polynomial{Rational{BigInt}, :h})

Polynomial{Rational{BigInt}, :h}

このように `Int` と `BigInt` の演算では `BigInt` の結果が得られるようになっている.